In [30]:
import os 
import json 

import boto3
import plotly.express as px
import pandas as pd

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [50]:
s3_client = boto3.client('s3')
bucket = 'mimic-log-odds-models'
prefix = 'movement-model-m3-a2-v1'

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

configs = {}
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('config.json'):
            config = json.loads(s3_client.get_object(Bucket=bucket, Key=obj['Key'])['Body'].read().decode('utf-8'))
            if 'model' in config:
                configs[config['run_id']] = config['model']
else:
    print("No objects found.")

In [86]:
results = (
    read_data_w_cache('select * from movement_model_experiment_m3_a2_v8')
    .sort_values(['run_id', 'epoch'], ascending=True)
)
results.head()

,loss,val_loss,epoch,experiment_name,run_id
426,0.438754,0.270166,1,movement-model-m3-a2-v8,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...
429,0.203300,0.201238,2,movement-model-m3-a2-v8,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...
430,0.178294,0.188623,3,movement-model-m3-a2-v8,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...
435,0.170615,0.183784,4,movement-model-m3-a2-v8,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...
436,0.166243,0.181022,5,movement-model-m3-a2-v8,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...


In [87]:
final = results.groupby('run_id')[['val_loss', 'loss']].last().reset_index().sort_values('val_loss', ascending=True)
final.head()

,run_id,val_loss,loss
21,ba11cc97ba31b37df6790f4a842654e309a9e11698fc11...,0.166877,0.151321
27,da5fb00097ba385ae981585ecaacb3de205dd681ae553e...,0.169256,0.151481
0,0807d8ed2985ece16eae4f7ebcbc32d988ec3f0d9332fd...,0.169590,0.148794
1,0c8d84a510c975c22dc8b24f266bb2b5026fc8b6edb0bd...,0.170109,0.151374
16,a8c7b9b1eef28d8689d88bdc38f2999c5352edfb01ab92...,0.172210,0.151136


In [53]:
px.scatter(final, x='loss', y='val_loss')

In [54]:
def count_layers(layers):
    layers = [int(l[1:]) for l in layers]
    return sum(layers)

config_df = pd.DataFrame([
    {
        'run_id': run_id, 
        'epochs': config['epochs'], 
        'batch_size': config['batch_size'],
        'neurons': count_layers(config['layers']),
        'depth': len(config['layers']),
        'layer1': int(config['layers'][0][1:]),
        'layer2': int(config['layers'][1][1:]),
    }
    for run_id, config in configs.items()
])
config_df.head()

,run_id,epochs,batch_size,neurons,depth,layer1,layer2
0,1173bd299a9ef2e8eb45f8d0bf2dced8ccfca3cb8828eb...,20,700,32,2,16,16
1,11d681fe4677486a52db6925a23b078a96f227e8ef0f0c...,20,600,28,3,8,16
2,1639888d0fec412d9ba059d271eee73a8070d4ac6acbf9...,25,500,40,3,16,16
3,18f2414f0df16919e21046a8cf98ea253001858f25e5b4...,25,700,32,2,16,16
4,29f2053d44b253a24a3bc6ef8bf4a454034ab753c30aaa...,15,600,32,2,16,16


In [55]:
df = final.merge(config_df)
px.scatter(df, x='epochs', y='val_loss')

In [56]:
px.scatter(df, x='batch_size', y='val_loss')

In [57]:
px.scatter(df, x='neurons', y='val_loss')

In [58]:
px.scatter(df, x='depth', y='val_loss')

In [59]:
px.scatter(df, x='layer1', y='val_loss')

In [60]:
px.scatter(df, x='layer2', y='val_loss')

In [88]:
x = final.sort_values('val_loss', ascending=True)['run_id'].values[0]
x

'ba11cc97ba31b37df6790f4a842654e309a9e11698fc11f6f719d86b83ba170b'

In [67]:
configs[x]

{'batch_size': 600, 'epochs': 20, 'layers': ['D16', 'D8']}

In [68]:
results[results['run_id'] == x]

,loss,val_loss,epoch,experiment_name,run_id
250,0.480688,0.304803,1,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
251,0.217460,0.200484,2,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
252,0.176773,0.188856,3,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
258,0.169519,0.186593,4,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
259,0.165525,0.182346,5,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
260,0.162226,0.180154,6,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
261,0.159958,0.176565,7,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
262,0.158184,0.175292,8,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
263,0.156665,0.173044,9,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
264,0.155796,0.172620,10,movement-model-m3-a2-v1,70d920d2d1269fd5ec235d7cc55091d1ab35158e3fe72e...
